Usando los métodos de Newton-Raphson y descenso del gradiente, encuentre la solución
de los siguientes sistemas de ecuaciones no lineales:

$$\ln(x_{1}^{2}+x_{2}^{2})-\sin(x_1x_2)=\ln(2)+\ln(\pi)$$

$$e^{x_1-x_2}+\cos(x_1x_2)=0$$

usando $x^{(0)}=(2,2)$

$$6x_1-2\cos(x_2x_3)-1=0$$

$$9x_2+\sqrt{x_1^{2}+\sin(x_3)+1.06}+0.9=0$$

$$60x_3+3e^{-x_1x_2}+10\pi-3=0$$

usando $x^{(0)}=(0,0,0)$

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

### Newton-Raphson

In [6]:
# sistema 1
G_1=(lambda x,y: np.log((x**2)+(y**2))-np.sin(x*y)-np.log(2)-np.log(np.pi), \
   lambda x,y: np.exp(x-y)+np.cos(x*y))

In [7]:
# sistema 2
G_2=(lambda x,y,z: 6*x - 2*np.cos(y*z) - 1, \
   lambda x,y,z: 9*y + np.sqrt((x**2)+np.sin(z)+1.06) + 0.9, \
   lambda x,y,z: 60*z + 3*np.exp(-x*y) + 10*np.pi- 3)

In [8]:
for i in range(2):
    print(G_1[i](2,2))

0.9983669705784182
0.34635637913638806


In [9]:
for i in range(3):
    print(G_2[i](0,0,0))

-3.0
1.9295630140987
31.41592653589793


In [10]:
def GetVectorF(G,r):
    
    dim = len(G)
    
    v = np.zeros(dim)
    if dim==1:
        for i in range(dim):
            v[i] = G[i](r[0])
    elif dim==2:
        for i in range(dim):
            v[i] = G[i](r[0],r[1])
    elif dim==3:
        for i in range(dim):
            v[i] = G[i](r[0],r[1],r[2])
    elif dim==4:
        for i in range(dim):
            v[i] = G[i](r[0],r[1],r[2],r[3])
    elif dim==5:
        for i in range(dim):
            v[i] = G[i](r[0],r[1],r[2],r[3],r[4])
    else:
        v=0
        print("Este sistema de ecuaciones es demasiado extenso :,(")
    return v

In [11]:
V_1=GetVectorF(G_1,(2,2))
V_2=GetVectorF(G_2,(0,0,0))

In [12]:
V_1,V_2

(array([0.99836697, 0.34635638]),
 array([-3.        ,  1.92956301, 31.41592654]))

In [13]:
def GetJacobian(G,r,h=1e-6):
    
    dim = len(G)
    
    J = np.zeros((dim,dim))
    if dim==1:
        for i in range(dim):
            J[i,0] = (  G[i](r[0]+h) - G[i](r[0]-h) )/(2*h)
    elif dim==2:
        for i in range(dim):
            J[i,0] = (  G[i](r[0]+h,r[1]) - G[i](r[0]-h,r[1]) )/(2*h)
            J[i,1] = (  G[i](r[0],r[1]+h) - G[i](r[0],r[1]-h) )/(2*h)
    elif dim==3:
        for i in range(dim):
            J[i,0] = (  G[i](r[0]+h,r[1],r[2]) - G[i](r[0]-h,r[1],r[2]) )/(2*h)
            J[i,1] = (  G[i](r[0],r[1]+h,r[2]) - G[i](r[0],r[1]-h,r[2]) )/(2*h)
            J[i,2] = (  G[i](r[0],r[1],r[2]+h) - G[i](r[0],r[1],r[2]-h) )/(2*h)
    elif dim==4:
        for i in range(dim):
            J[i,0] = (  G[i](r[0]+h,r[1],r[2],r[3]) - G[i](r[0]-h,r[1],r[2],r[3]) )/(2*h)
            J[i,1] = (  G[i](r[0],r[1]+h,r[2],r[3]) - G[i](r[0],r[1]-h,r[2],r[3]) )/(2*h)
            J[i,2] = (  G[i](r[0],r[1],r[2]+h,r[3]) - G[i](r[0],r[1],r[2]-h,r[3]) )/(2*h)
            J[i,3] = (  G[i](r[0],r[1],r[2],r[3]+h) - G[i](r[0],r[1],r[2],r[3]-h) )/(2*h)
    elif dim==5:
        for i in range(dim):
            J[i,0] = (  G[i](r[0]+h,r[1],r[2],r[3],r[4]) - G[i](r[0]-h,r[1],r[2],r[3],r[4]) )/(2*h)
            J[i,1] = (  G[i](r[0],r[1]+h,r[2],r[3],r[4]) - G[i](r[0],r[1]-h,r[2],r[3],r[4]) )/(2*h)
            J[i,2] = (  G[i](r[0],r[1],r[2]+h,r[3],r[4]) - G[i](r[0],r[1],r[2]-h,r[3],r[4]) )/(2*h)
            J[i,3] = (  G[i](r[0],r[1],r[2],r[3]+h,r[4]) - G[i](r[0],r[1],r[2],r[3]-h,r[4]) )/(2*h)
            J[i,4] = (  G[i](r[0],r[1],r[2],r[3],r[4]+h) - G[i](r[0],r[1],r[2],r[3],r[4]-h) )/(2*h)
    else:
        J=0
        print("wey ya")
    return J.T

In [14]:
def NewtonRaphson(G,r,error=1e-10):
    
    it = 0
    d = 1
    Vector_d = np.array([])
    
    while d > error:
        
        it += 1
        
        rc = r
        
        F = GetVectorF(G,r)
        J = GetJacobian(G,r)
        InvJ = np.linalg.inv(J)
        
        r = rc - np.dot( InvJ, F )
        
        diff = r - rc
        
        d = np.linalg.norm(diff)
        
        Vector_d = np.append( Vector_d , d )
        
    return r,it,Vector_d

In [15]:
res_1,it_1,distancias_1 = NewtonRaphson(G_1,[2,2])
res_2,it_2,distancias_2 = NewtonRaphson(G_2,[0,0,0])

In [16]:
print(res_1,it_1)
print(res_2,it_2)

[1.77245385 1.77245385] 109
[ 0.49814468 -0.1996059  -0.52882598] 12


In [17]:
np.round(GetVectorF(G_1,res_1)),np.round(GetVectorF(G_2,res_2))

(array([-0.,  0.]), array([-0.,  0.,  0.]))

### Descenso de gradiente

In [18]:
def GetMetric(G,r):
    v = GetVectorF(G,r)
    return 0.5*np.linalg.norm(v)**2

In [19]:
M_1=GetMetric(G_1,[2,2])
M_2=GetMetric(G_2,[0,0,0])

In [20]:
M_1,M_2

(0.5583496746551987, 499.8418267671567)

In [21]:
def GetFig(F,R,it):
    
    fig = plt.figure(figsize=(8,4))
    
    labels = ['X','Y','Z']
    
    ax = fig.add_subplot(1,2,1)
    ax1 = fig.add_subplot(1,2,2)

    ax.set_title('Metric: %.20f' %(F[it]))

    ax.plot(F[:it])
    ax.set_xlabel('%.0f' %(it))
    ax.set_yscale('log')
    ax1.plot(R[:it],label=labels)
    ax1.set_xlabel('%.0f' %(it))
    ax1.legend(loc=0)
    
    plt.show()

In [22]:
def GetSolve_3(G,r,lr=1e-3,epochs=int(1e5),error=1e-7):
    
    d = 1
    it = 0
    Vector_F = np.array([])
    
    R_vector = np.array(r)
    
    while d > error and it < epochs:
        
        CurrentF = GetMetric(G,r)
        
        J = GetJacobian(G,r)
        
        GVector = GetVectorF(G,r)
        
        #Machine Learning
        r -= lr*np.dot(J,GVector) 
        
        R_vector = np.vstack((R_vector,r))
        
        NewF = GetMetric(G,r)
        
        
        Vector_F = np.append(Vector_F,NewF)
        
        d = np.abs( CurrentF - NewF )/NewF
        
        
        """"if it%500 == 0:
            
            #print(it,d)
            clear_output(wait=True)
            GetFig(Vector_F,R_vector,it)
            time.sleep(0.01)"""
            
        it += 1
        
    if d < error:
        print(' Entrenamiento completo ', d, 'iteraciones', it)
        
    if it == epochs:
        print(' Entrenamiento no completado ')
        
    return r,it,Vector_F,R_vector

In [27]:
xsol1,it1,F1,R1 = GetSolve_3(G_2,[0,0,0],lr=1e-4)
xsol2,it2,F2,R2 = GetSolve_3(G_1,[2,2],lr=1e-3)
xsol1,xsol2

 Entrenamiento completo  0.0 iteraciones 8536
 Entrenamiento completo  0.0 iteraciones 13218


(array([ 0.49814468, -0.1996059 , -0.52882598]),
 array([1.77245385, 1.77245385]))